In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

2025-05-29 00:51:40.556968: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-29 00:51:40.560971: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-29 00:51:40.570728: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748469100.587061   25736 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748469100.591244   25736 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748469100.604438   25736 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

# Specify each path

In [2]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.keras'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [3]:
NUM_CLASSES = 9

# Dataset reading

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

2025-05-29 00:51:43.493369: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 9)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,169 (4.57 KB)

 Trainable params: 1,169 (4.57 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [10]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [11]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
45/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1828 - loss: 2.2183 
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1834 - loss: 2.2160 - val_accuracy: 0.2130 - val_loss: 2.0482
Epoch 2/1000
43/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2230 - loss: 2.0444
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2239 - loss: 2.0421 - val_accuracy: 0.2317 - val_loss: 1.9458
Epoch 3/1000
27/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2440 - loss: 1.9725 
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2519 - loss: 1.9619 - val_accuracy: 0.4000 - val_loss: 1.8538
Epoch 4/1000
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2757 - loss: 1.8926 
Epoch 4: saving model to model/keypoint_classifier/keypoint_classifier.keras


46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5430 - loss: 1.1592 - val_accuracy: 0.7990 - val_loss: 0.8079
Epoch 25/1000
35/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5553 - loss: 1.1284 
Epoch 25: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5539 - loss: 1.1345 - val_accuracy: 0.7875 - val_loss: 0.7915
Epoch 26/1000
33/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5489 - loss: 1.1307 
Epoch 26: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5504 - loss: 1.1334 - val_accuracy: 0.8130 - val_loss: 0.7777
Epoch 27/1000
27/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5625 - loss: 1.1230 
Epoch 27: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5638 - loss: 1.1217 - val_accuracy: 0.7948 - val_loss: 0.7653
Epoch 28/1000
28/46 ━━━━━━━━━━━━━━━━━━━━ 0

28/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6145 - loss: 1.0497 
Epoch 48: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6193 - loss: 1.0358 - val_accuracy: 0.8608 - val_loss: 0.6077
Epoch 49/1000
27/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6268 - loss: 0.9730 
Epoch 49: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6257 - loss: 0.9784 - val_accuracy: 0.8618 - val_loss: 0.6007
Epoch 50/1000
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6273 - loss: 1.0141
Epoch 50: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6253 - loss: 1.0120 - val_accuracy: 0.8571 - val_loss: 0.6034
Epoch 51/1000
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6198 - loss: 1.0079
Epoch 51: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6381 - loss: 0.9582 - val_accuracy: 0.8857 - val_loss: 0.5376
Epoch 72/1000
37/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6570 - loss: 0.9375
Epoch 72: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6555 - loss: 0.9370 - val_accuracy: 0.8868 - val_loss: 0.5363
Epoch 73/1000
34/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6358 - loss: 0.9648
Epoch 73: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6374 - loss: 0.9601 - val_accuracy: 0.8904 - val_loss: 0.5336
Epoch 74/1000
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6416 - loss: 0.9472 
Epoch 74: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6460 - loss: 0.9444 - val_accuracy: 0.8826 - val_loss: 0.5371
Epoch 75/1000
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 

28/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6734 - loss: 0.9156 
Epoch 95: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6704 - loss: 0.9186 - val_accuracy: 0.8935 - val_loss: 0.5029
Epoch 96/1000
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6468 - loss: 0.9116 
Epoch 96: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6511 - loss: 0.9079 - val_accuracy: 0.8914 - val_loss: 0.5003
Epoch 97/1000
32/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6516 - loss: 0.9248 
Epoch 97: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6531 - loss: 0.9217 - val_accuracy: 0.8940 - val_loss: 0.5003
Epoch 98/1000
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6523 - loss: 0.9219 
Epoch 98: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6524 - loss: 0.8902 - val_accuracy: 0.8982 - val_loss: 0.4797
Epoch 119/1000
32/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6633 - loss: 0.9071 
Epoch 119: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6633 - loss: 0.9086 - val_accuracy: 0.8914 - val_loss: 0.4897
Epoch 120/1000
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6738 - loss: 0.8942 
Epoch 120: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6731 - loss: 0.8946 - val_accuracy: 0.8940 - val_loss: 0.4875
Epoch 121/1000
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6687 - loss: 0.8943 
Epoch 121: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6687 - loss: 0.8932 - val_accuracy: 0.8914 - val_loss: 0.4802
Epoch 122/1000
27/46 ━━━━━━━━━━━━━━━

Epoch 142/1000
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6780 - loss: 0.8723 
Epoch 142: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6736 - loss: 0.8859 - val_accuracy: 0.8945 - val_loss: 0.4797
Epoch 143/1000
28/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6619 - loss: 0.9284 
Epoch 143: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6612 - loss: 0.9236 - val_accuracy: 0.8899 - val_loss: 0.4837
Epoch 144/1000
26/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6907 - loss: 0.8319 
Epoch 144: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6847 - loss: 0.8484 - val_accuracy: 0.8925 - val_loss: 0.4778
Epoch 145/1000
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6771 - loss: 0.8808 
Epoch 145: saving model to model/keypoint_classifier/keypoint_

22/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6770 - loss: 0.8878 
Epoch 165: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6771 - loss: 0.8794 - val_accuracy: 0.9003 - val_loss: 0.4676
Epoch 166/1000
32/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6573 - loss: 0.9120 
Epoch 166: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6593 - loss: 0.9068 - val_accuracy: 0.8956 - val_loss: 0.4782
Epoch 167/1000
25/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6761 - loss: 0.9130 
Epoch 167: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6788 - loss: 0.9012 - val_accuracy: 0.9029 - val_loss: 0.4714
Epoch 168/1000
31/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6749 - loss: 0.8472 
Epoch 168: saving model to model/keypoint_classifier/keypoint_classifier.kera

46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6902 - loss: 0.8357 - val_accuracy: 0.8868 - val_loss: 0.4651
Epoch 189/1000
33/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6926 - loss: 0.8464 
Epoch 189: saving model to model/keypoint_classifier/keypoint_classifier.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6884 - loss: 0.8542 - val_accuracy: 0.8888 - val_loss: 0.4706
Epoch 189: early stopping


In [12]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8995 - loss: 0.4460 


In [13]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [14]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
[1.1524495e-02 2.4923788e-02 1.2290637e-02 6.5973632e-05 3.1461304e-01
 1.9635454e-04 8.3312439e-04 4.2017698e-03 6.3135082e-01]
8


# Confusion matrix

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

ModuleNotFoundError: No module named 'pandas'

# Convert to model for Tensorflow-Lite

In [ ]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [ ]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

# Inference test

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [ ]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [ ]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))